In [1]:
import pygame
import math
from queue import PriorityQueue
import time
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
VISIT1 = (0, 255, 230)
VISIT2 = (0, 80, 230)
LOOK = (0, 255, 164)
START = (217, 1, 183)
END = RED
GREY = (63, 63, 63)
OPEN = (0, 255, 149)
PATH1 = (255, 200, 0)
PATH2 = (255, 253, 0)
VISITED = []

In [3]:
class Node:
    def __init__(self, row, col, width, total_rows):
        self.last = pygame.time.get_ticks()
        self.cooldown = 300 
        self.row = row
        self.col = col
        self.x = row*width
        self.y = col*width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
        self.dec_animation = False
        
    def get_pos(self):
        return self.row, self.col
    
    def is_visited(self):
        return self.color == VISIT1
    
    def is_open(self):
        return self.color == OPEN
    
    def is_barrier(self):
        return self.color == BLACK
    
    def is_start(self):
        return self.color == START
    
    def is_end(self):
        return self.color == END
    
    def is_neutral(self):
        return self.color == WHITE
    
    def is_looked(self):
        return self.color == LOOK
    
    def reset(self):
        self.color = WHITE
    
    def make_visit(self):
        self.color = VISIT2
        
    def make_open(self):
        self.color = OPEN
    
    def make_barrier(self):
        if not self.is_start() and not self.is_end():
            self.color = BLACK
    
    def make_end(self):
        self.color = END
    
    def make_start(self):
        self.color = START
        
    def make_path(self):
        if not self.is_start():
            self.color = PATH1
    
    def looking_at(self):
        self.color = LOOK
    
    def draw(self, win):
        try:
            pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))
        except:
            print(self.color)
            print(self.row, self.col)

    def update_neighbors(self, grid):
        r = self.row 
        c = self.col
        if r < self.total_rows-1 and not grid[r+1][c].is_barrier():
            self.neighbors.append(grid[r+1][c])
            
        if r > 0 and not grid[r-1][c].is_barrier():
            self.neighbors.append(grid[r-1][c])
            
        if c < self.total_rows-1 and not grid[r][c+1].is_barrier():
            self.neighbors.append(grid[r][c+1])
            
        if c > 0 and not grid[r][c-1].is_barrier():
            self.neighbors.append(grid[r][c-1])
    
    def __lt__(self, other):
        return False


In [4]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

In [5]:
def visit_animation(visited):
    for node in visited:
        if node.color == VISIT1:
            visited.remove(node)
            continue
        r, g, b = node.color
        g += 1
        node.color = (r, g, b)

In [6]:
def path_animation(path):
    for node in path:
        if not node.is_start():
            r, g, b = node.color
            if node.dec_animation:
                g -= 1
                if g <= PATH1[1]:
                    node.dec_animation = False
            else:
                g += 1
                if g >= PATH2[1]:
                    node.dec_animation = True
        for i in range(1000):
            i
        node.color = (r, g, b)

In [7]:
pygame.time.wait(600)
print("Loops")

Loops


In [8]:
# def looking_at(current):
#     clock = pygame.time.Clock()
#     FPS = 60
#     change_every_x_seconds = 0.1
#     number_of_steps = change_every_x_seconds * FPS
#     step = 1
#     current.color = (227, 255, 0)
#     r, g, b = current.color
#     while r > 38 and g > 142 and b < 191:
#         print(current.color)
#         step += 1
#         if step < number_of_steps:
#             r, g, b = [x + (((y-x)/number_of_steps)*step) for x, y in zip(LOOK, VISIT2)]
#             current.color = (r, g, b)
#         current.draw(WIN)
#         pygame.display.update()
#         clock.tick(FPS)

In [9]:
def reconstruct_path(came_from, start, current, draw, visited):
    path = []
    while current in came_from:
        current = came_from[current]
        if current in visited:
            visited.remove(current)
        if current != start:
            path.insert(0, current)
        current.make_path()
#         path_animation(path)
        draw()
    return path

In [10]:
def algorithm(draw, grid, start, end):
    count = 0
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {node: float("inf") for row in grid for node in row}
    g_score[start] = 0
    f_score = {node: float("inf") for row in grid for node in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    visited = []
    nebrs = []
    
    open_set_hash = {start}
    
    while not open_set.empty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        
        current = open_set.get()[2]
        open_set_hash.remove(current)
        
        if current == end:
            path = reconstruct_path(came_from, start, end, draw, visited)
            start.make_start()
            return visited, path
                
        for neighbor in current.neighbors:
            if not neighbor.is_barrier():
                temp_g_score = g_score[current] + 1
                if temp_g_score < g_score[neighbor]:

                    if neighbor not in open_set_hash:
                        came_from[neighbor] = current
                        g_score[neighbor] = temp_g_score
                        f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                        count+=1
                        open_set.put((f_score[neighbor], count, neighbor))
                        open_set_hash.add(neighbor)
                        if neighbor != end:
                            nebrs.append(neighbor)
                            neighbor.make_open()
        
        if current != start:
            visited.append(current)
            current.make_visit()
        
        visit_animation(visited)
#         nebr_animation(nebrs)
        draw()
            
    return visited, False

In [11]:
path = False
if path:
    print(1)

In [12]:
def is_free(grid, x, y):
    count = 0
    if y+1 < 50 and grid[x][y+1].is_barrier() :
        count +=1
    if y-1>=0 and grid[x][y-1].is_barrier():
        count +=1
    if x+1 < 50 and grid[x+1][y].is_barrier():
        count+=1
    if x-1>=0 and grid[x-1][y].is_barrier():
        count+=1
    if count >= 3:
        return True
    return False

In [13]:
def unvisited_n(grid, x, y):
    n = []
    if y+1 < len(grid) and grid[x][y+1].is_barrier() and is_free(grid, x, y+1):
        n.append((x, y+1))
    if y-1>=0 and grid[x][y-1].is_barrier() and is_free(grid, x, y-1):
        n.append((x, y-1))
    if x+1 < len(grid) and grid[x+1][y].is_barrier() and is_free(grid, x+1, y):
        n.append((x+1, y))
    if x-1>=0 and grid[x-1][y].is_barrier() and is_free(grid, x-1, y):
        n.append((x-1, y))
    return n

In [14]:
def make_black(grid):
    for row in grid:
        for node in row:
            node.make_barrier()

In [15]:
def dfs_maze(draw, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid)
    x, y = 1, 1
    head = grid[x][y]
    head.looking_at()
    stack = [(x, y)]
    while True:
        neighbors = unvisited_n(grid, x, y)
        if len(neighbors) > 0:
            random_index = np.random.randint(len(neighbors))
            x, y = neighbors[random_index]
            head = grid[x][y]
            head.looking_at()
            stack.append((x, y))
            draw()
        else:
            if len(stack) > 0:
                x, y = stack.pop()
                grid[x][y].reset()
            if len(stack) > 0:
                x, y = stack[-1]
                draw()
            else:
                break

In [16]:
def is_open_up(grid, li):
    n = []
    for x, y in li:
        if grid[x][y-1].is_neutral():
            n.append(grid[x][y])
    if len(n)>0:
        node = np.random.choice(n)
        node.reset()

In [17]:
def sidewinder_maze(draw, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid)
    for x in range(1, right-1):
        grid[x][1].reset()
        draw()
    for y in range(3, bottom, 2):
        x = 1
        next_row = False
        while not next_row:
            lim = x+5
            if lim >= right-1:
                x_lim = right-1
                next_row = True
            else:
                x_lim = np.random.randint(x+2, lim)
            li = []
            
            for i in range(x, x_lim):
                li.append((i, y-1))
                grid[i][y].reset()
                draw()
            is_open_up(grid, li)
            draw()
            x = x_lim+1
    for x in range(1, right-1):
        grid[x][0].make_barrier()
        draw()

In [18]:
def moded_sidewinder_maze(draw, grid, start, end, left, right, top, bottom, win, vertical=True):
    make_black(grid)
    for x in range(1, right-1):
        grid[x][0].reset()
        draw()
    for y in range(2, bottom, 2):
        x = 1
        next_row = False
        while not next_row:
            lim = x+5
            if lim >= right-1:
                x_lim = right-1
                next_row = True
            else:
                x_lim = np.random.randint(x+2, lim)
            li = []
            
            for i in range(x, x_lim):
                li.append((i, y-1))
                grid[i][y].reset()
                draw()
            if (y == 2 and not np.random.randint(0, 5)) or (not next_row and not np.random.randint(0, 5)):
                grid[x_lim][y].reset()
                draw()
            is_open_up(grid, li)
            draw()
            x = x_lim+1
    for x in range(1, right-1):
        grid[x][0].make_barrier()
        draw()

In [19]:
def recursive_maze(draw, grid, start, end, left, right, top, bottom):
    if right-left >= 1 and bottom-top >=1:
#         print(left, right, top, bottom)
        vertical = True
        test = False
        if right-left >= bottom-top:
            if bottom-top == 1:
                if (bottom < len(grid) and grid[left][bottom].is_barrier()) and (top-1 >= 0 and grid[left][top-1].is_barrier()):
                    vertical = False
            if vertical == True:
                l = left
                r = right
                if left-1>=0 and grid[left-1][top].is_barrier():
                    l = left+1
                if right<50 and grid[right][top].is_barrier():
                    r = right-1
                if l >= r:
                    vertical = False 
            if vertical == True:
                test = True
#                 print("--->", l, r)
                rand = np.random.randint(l, r)
                if top+1 < bottom-1:
                    br = np.random.randint(top+1, bottom-1)
                elif top+1 == bottom-1:
                    br = top+1
                else:
                    br = bottom
                
        elif right-left <= bottom-top:
            if right-left == 1:
                if (right < len(grid) and grid[right][top].is_barrier()) and (left-1 >= 0 and grid[left-1][top].is_barrier()):
                    return 
            t = top
            b = bottom
            if top-1>= 0 and grid[left][top-1].is_barrier():
                t = top+1
            if bottom < 50 and grid[left][bottom].is_barrier():
                b = bottom-1
            if t >= b:
                return
            test = True
#             print("--->", t, b)
            rand = np.random.randint(t, b)
            if left+1 < right-1:
                br = np.random.randint(left+1, right-1)
            elif left+1 == right-1:
                br = left+1
            else:
                br = right
            vertical = False
        if test == False:
            return
        if vertical:
            for y in range(top, br):
#                 print(rand, y)
                grid[rand][y].make_barrier()
                draw()
            for y in range(br+1, bottom):
#                 print(rand, y)
                grid[rand][y].make_barrier()
                draw()
        else:
            for x in range(left, br):
                print(x, rand)
                grid[x][rand].make_barrier()
                draw()
            for x in range(br+1, right):
                print(x, rand)
                grid[x][rand].make_barrier()
                draw()
        if vertical:
            recursive_maze(draw, grid, start, end, rand+1, right, top, br)
            recursive_maze(draw, grid, start, end, left, rand, top, br)
            recursive_maze(draw, grid, start, end, rand+1, right, br+1, bottom)
            recursive_maze(draw, grid, start, end, left, rand, br+1, bottom)
        else:
            recursive_maze(draw, grid, start, end, left, br, top, rand)
            recursive_maze(draw, grid, start, end, left, br, rand+1, bottom)
            recursive_maze(draw, grid, start, end, br+1, right, top, rand)
            recursive_maze(draw, grid, start, end, br+1, right, rand+1, bottom)

In [20]:
def recursive_div(draw, grid, start, end, left, right, top, bottom, break_at = 0):
    for i in range(right):
        grid[0][i].make_barrier()
        grid[bottom-1][right-1-i].make_barrier()
        draw()
    for i in range(bottom-1):
        grid[i][0].make_barrier()
        grid[bottom-1-i][right-1].make_barrier()
        draw()
    recursive_maze(draw, grid, start, end, left+1, right-1, top+1, bottom-1)

In [21]:
def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(i, j, gap, rows)
            grid[i].append(node)
    return grid

In [22]:
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i*gap), (width, i*gap))
    for i in range(rows):
        pygame.draw.line(win, GREY, (i*gap, 0), (i*gap, width))

In [23]:
def draw(win, grid, rows, width):
    win.fill(WHITE)
    for row in grid:
        for node in row:
            node.draw(win)
    draw_grid(win, rows, width)
    pygame.display.update()

In [24]:
def get_clicked_pos(pos, rows, width):
    gap = width//rows
    y, x = pos
    row = y // gap
    col = x // gap
    return row, col

In [25]:
def main(win, width):
    ROWS = 50
    grid = make_grid(ROWS, width)
    
    start = None
    end = None
    
    run = True
    started = False
    visited = []
    path = False
    while run:
        if len(visited):
            visit_animation(visited)
            
        if path:
            path_animation(path)
            
        draw(win, grid, ROWS, width)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                
            if started:
                continue
            
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, ROWS, width)
                node = grid[row][col]
                if path:
                    if node in path:
                        path.remove(node)
                if not start and node != end:
                    start = node
                    start.make_start()
                elif not end and node != start:
                    end = node
                    end.make_end()
                
                elif node != end and node != start:
                    node.make_barrier()
                    
            elif pygame.mouse.get_pressed()[2]:
                pos = pygame.mouse.get_pos()
#                 print(pos)
                row,col = get_clicked_pos(pos, ROWS, width)
                node = grid[row][col]
                if path:
                    if node in path:
                        path.remove(node)
                node.reset()
                if node == start:
                    start = None
                elif node == end:
                    end = None
            
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and not started:
                    if start and end:
                        for row in grid:
                            for node in row:
                                node.update_neighbors(grid)
                                if not node.is_neutral() and node != start and node != end and not node.is_barrier():
                                    node.reset()
                        visited = []
                        path = []
                        visited, path = algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)
            
                if event.key == pygame.K_LEFT and not started:
                    start = None
                    end = None
                    visited = []
                    path = []
                    for row in grid:
                        for node in row:
                            if node.is_barrier() or node.is_start() or node.is_end():
                                node.reset()
                    dfs_maze(lambda: draw(win, grid, ROWS, width), grid, start, end, 0, ROWS, 0, ROWS, win)
                
                if event.key == pygame.K_UP and not started:
                    start = None
                    end = None
                    visited = []
                    path = []
                    for row in grid:
                        for node in row:
                            if node.is_barrier() or node.is_start() or node.is_end():
                                node.reset()
                    moded_sidewinder_maze(lambda: draw(win, grid, ROWS, width), grid, start, end, 0, ROWS, 0, ROWS, win)
                
                if event.key == pygame.K_DOWN and not started:
                    start = None
                    end = None
                    visited = []
                    path = []
                    for row in grid:
                        for node in row:
                            node.reset()
                    recursive_div(lambda: draw(win, grid, ROWS, width), grid, start, end, 0, ROWS, 0, ROWS)
                
                if event.key == pygame.K_c and not started:
                    start = None
                    end = None
                    visited = []
                    path = []
                    for row in grid:
                        for node in row:
                            node.reset()
            
    pygame.quit()

In [27]:
WIDTH = 750
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("A* Path Finding Algorithm")
main(WIN, WIDTH)

1 49 1 49
---> 2 48
25 1
25 2
25 3
25 4
25 5
25 6
25 7
25 8
25 9
25 10
25 11
25 12
25 13
25 14
25 15
25 16
25 17
25 18
25 19
25 20
25 21
25 22
25 23
25 24
25 25
25 26
25 27
25 28
25 29
25 30
25 31
25 32
25 34
25 35
25 36
25 37
25 38
25 39
25 40
25 41
25 42
25 43
25 44
25 45
25 46
25 47
25 48
26 49 1 33
---> 2 33
26 5
27 5
28 5
29 5
30 5
32 5
33 5
34 5
35 5
36 5
37 5
38 5
39 5
40 5
41 5
42 5
43 5
44 5
45 5
46 5
47 5
48 5
26 31 1 5
---> 27 31
27 1
27 2
27 4
28 31 1 3
---> 29 31
29 1
30 31 1 2
28 29 1 2
26 27 1 3
28 31 4 5
---> 29 31
29 4
30 31 4 5
28 29 4 5
26 27 4 5
26 31 6 33
---> 7 33
26 15
27 15
29 15
30 15
26 28 6 15
---> 7 14
26 12
26 27 6 12
---> 7 11
26 7
26 27 6 7
26 27 8 12
---> 9 11
26 10
26 27 8 10
26 27 11 12
26 27 13 15
26 28 16 33
---> 17 33
26 24
26 27 16 24
---> 17 23
26 19
26 27 16 19
---> 17 18
26 17
26 27 16 17
26 27 18 19
26 27 20 24
---> 21 23
26 22
26 27 20 22
26 27 23 24
26 27 25 33
---> 26 33
26 29
26 27 25 29
---> 26 28
26 26
26 27 25 26
26 27 27 29
26 27 30 33


22 23 8 12
---> 9 11
22 10
22 23 8 10
22 23 11 12
22 23 13 15
20 23 16 19
---> 20 23
20 16
20 18
21 23 16 17
---> 22 23
22 16
21 22 16 17
21 23 18 19
---> 22 23
22 18
21 22 18 19
17 23 20 21
15 23 22 33
---> 23 33
15 24
16 24
17 24
18 24
19 24
21 24
22 24
15 20 22 24
---> 15 20
19 22
15 19 22 23
---> 15 18
16 22
17 19 22 23
15 16 22 23
15 20 25 33
---> 26 33
15 31
16 31
18 31
19 31
15 17 25 31
---> 26 30
15 27
15 16 25 27
15 16 28 31
---> 29 30
15 29
15 16 28 29
15 16 30 31
15 17 32 33
---> 15 17
15 32
16 17 32 33
18 20 25 31
---> 26 30
18 28
18 19 25 28
---> 26 27
18 26
18 19 25 26
18 19 27 28
18 19 29 31
18 20 32 33
---> 18 20
19 32
18 19 32 33
21 23 22 24
---> 21 23
21 22
22 23 22 23
21 23 25 33
---> 26 33
21 31
21 22 25 31
---> 26 30
21 27
21 22 25 27
21 22 28 31
---> 29 30
21 29
21 22 28 29
21 22 30 31
21 22 32 33
---> 21 22
21 32
24 25 1 3
24 25 4 33
---> 5 33
24 7
24 25 4 7
---> 5 6
24 5
24 25 4 5
24 25 6 7
24 25 8 33
---> 9 33
24 15
24 25 8 15
---> 9 14
24 11
24 25 8 11
---> 9 

27 28 15 16
27 28 17 31
---> 18 30
27 23
27 28 17 23
---> 18 22
27 18
27 28 17 18
27 28 19 23
---> 20 22
27 21
27 28 19 21
27 28 22 23
27 28 24 31
---> 25 30
27 27
27 28 24 27
---> 25 26
27 25
27 28 24 25
27 28 26 27
27 28 28 31
---> 29 30
27 29
27 28 28 29
27 28 30 31
27 28 32 38
---> 33 37
27 36
27 28 32 36
---> 33 35
27 34
27 28 32 34
27 28 35 36
27 28 37 38
29 32 15 31
---> 16 30
29 28
31 28
29 30 15 28
---> 16 27
29 17
29 30 15 17
29 30 18 28
---> 19 27
29 21
29 30 18 21
---> 19 20
29 19
29 30 18 19
29 30 20 21
29 30 22 28
---> 23 27
29 25
29 30 22 25
---> 23 24
29 23
29 30 22 23
29 30 24 25
29 30 26 28
29 30 29 31
31 32 15 28
---> 16 27
31 21
31 32 15 21
---> 16 20
31 18
31 32 15 18
---> 16 17
31 16
31 32 15 16
31 32 17 18
31 32 19 21
31 32 22 28
---> 23 27
31 23
31 32 22 23
31 32 24 28
---> 25 27
31 25
31 32 24 25
31 32 26 28
31 32 29 31
29 32 32 38
---> 33 37
29 36
31 36
29 30 32 36
---> 33 35
29 33
29 30 32 33
29 30 34 36
29 30 37 38
31 32 32 36
---> 33 35
31 33
31 32 32 33
31

8 36
9 36
10 36
11 36
12 36
1 2 24 36
---> 25 35
1 32
1 2 24 32
---> 25 31
1 28
1 2 24 28
---> 25 27
1 26
1 2 24 26
1 2 27 28
1 2 29 32
---> 30 31
1 30
1 2 29 30
1 2 31 32
1 2 33 36
---> 34 35
1 34
1 2 33 34
1 2 35 36
1 2 37 44
---> 38 43
1 39
1 2 37 39
1 2 40 44
---> 41 43
1 41
1 2 40 41
1 2 42 44
3 13 24 36
---> 25 35
3 26
4 26
5 26
6 26
7 26
8 26
9 26
11 26
12 26
3 10 24 26
---> 3 10
7 24
8 10 24 25
---> 9 10
9 24
8 9 24 25
3 7 24 25
---> 3 6
4 24
5 7 24 25
3 4 24 25
3 10 27 36
---> 28 35
3 30
4 30
5 30
7 30
8 30
9 30
3 6 27 30
---> 3 6
3 27
3 29
4 6 27 28
---> 5 6
5 27
4 5 27 28
4 6 29 30
---> 5 6
5 29
4 5 29 30
3 6 31 36
---> 32 35
3 34
5 34
3 4 31 34
---> 32 33
3 32
3 4 31 32
3 4 33 34
3 4 35 36
5 6 31 34
---> 32 33
5 32
5 6 31 32
5 6 33 34
5 6 35 36
7 10 27 30
---> 7 10
8 27
8 29
9 10 27 28
7 8 27 28
9 10 29 30
7 8 29 30
7 10 31 36
---> 32 35
7 34
9 34
7 8 31 34
---> 32 33
7 32
7 8 31 32
7 8 33 34
7 8 35 36
9 10 31 34
---> 32 33
9 32
9 10 31 32
9 10 33 34
9 10 35 36
11 13 24 26
